<a href="https://colab.research.google.com/github/hklfach/Computer-Vision/blob/main/Assignment4_1301202398_Muhamad_Fachri_Haikal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1301202398 - Muhamad Fachri Haikal

Perbandingan hasil klasifikasi data Cats vs Dogs menggunakan ekstraksi fitur HOG, LBP, dan SIFT

### Library

In [ ]:
import cv2
import numpy as np
import os
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

### Download Dataset Cats vs Dogs

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs_filtered.zip
    
!unzip -q cats_and_dogs_filtered.zip

--2023-04-13 15:03:59--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  29.8MB/s    in 2.2s    

2023-04-13 15:04:01 (29.8 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



### Read Data

In [ ]:
def read_images(path, label):
    images = []
    labels = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path, filename))
        img = cv2.resize(img, (150, 150))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

# read images and labels
cat_path = "cats_and_dogs_filtered/train/cats"
dog_path = "cats_and_dogs_filtered/train/dogs"
cat_images, cat_labels = read_images(cat_path, 0)
dog_images, dog_labels = read_images(dog_path, 1)

# concatenate images and labels
images = np.concatenate((cat_images, dog_images))
labels = np.concatenate((cat_labels, dog_labels))

### Feature Extraction

In [ ]:
def extract_HOG(img):
    hog = cv2.HOGDescriptor()
    return hog.compute(img)

def extract_LBP(img):
    lbp = local_binary_pattern(img, 8, 2, method='default')
    return lbp.flatten()

def extract_SIFT(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    desc_samples = descriptors[np.random.randint(descriptors.shape[0], size=50)]
    return desc_samples

# extract features
features_HOG = [extract_HOG(img) for img in images]
features_LBP = [extract_LBP(img) for img in images]
features_SIFT = [extract_SIFT(img) for img in images]

### Create Model with HOG



In [37]:
# split dataset into training and testing sets
X_train_HOG, X_test_HOG, y_train_HOG, y_test_HOG = train_test_split(features_HOG, labels, test_size=0.2, random_state=42)

# train and test using linear SVM
svm_HOG = SVC()
svm_HOG.fit(X_train_HOG, y_train_HOG)

y_pred_HOG = svm_HOG.predict(X_test_HOG)

In [38]:
print(classification_report(y_test_HOG, y_pred_HOG))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73       199
           1       0.73      0.75      0.74       201

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



### Create Model with LBP

In [ ]:
# split dataset into training and testing sets
X_train_LBP, X_test_LBP, y_train_LBP, y_test_LBP = train_test_split(features_LBP, labels, test_size=0.2, random_state=42)

# train and test using linear SVM
svm_LBP = SVC()
svm_LBP.fit(X_train_LBP, y_train_LBP)

y_pred_LBP = svm_LBP.predict(X_test_LBP)

In [ ]:
print(classification_report(y_test_LBP, y_pred_LBP))

              precision    recall  f1-score   support

           0       0.57      0.50      0.53       199
           1       0.56      0.63      0.59       201

    accuracy                           0.56       400
   macro avg       0.57      0.56      0.56       400
weighted avg       0.57      0.56      0.56       400



### Create Model with SIFT

In [ ]:
# split dataset into training and testing sets
X_train_SIFT, X_test_SIFT, y_train_SIFT, y_test_SIFT = train_test_split(features_SIFT, labels, test_size=0.2, random_state=42)

X_train_SIFT_1d = []
for data in X_train_SIFT:
  X_train_SIFT_1d.append(np.ravel(data))

X_test_SIFT_1d = []
for data in X_test_SIFT:
  X_test_SIFT_1d.append(np.ravel(data))

# train and test using linear SVM
svm_SIFT = SVC()
svm_SIFT.fit(X_train_SIFT_1d, y_train_SIFT)

y_pred_SIFT = svm_SIFT.predict((X_test_SIFT_1d))

In [ ]:
print(classification_report(y_test_SIFT, y_pred_SIFT))

              precision    recall  f1-score   support

           0       0.61      0.57      0.59       199
           1       0.60      0.63      0.62       201

    accuracy                           0.60       400
   macro avg       0.60      0.60      0.60       400
weighted avg       0.60      0.60      0.60       400



### Analisis

Dalam hasil analisis ini, penggunaan ekstraksi fitur HOG menghasilkan akurasi yang lebih tinggi daripada ekstraksi fitur SIFT dan LBP untuk klasifikasi gambar kucing dan anjing menggunakan model SVM. Hal ini dapat disebabkan oleh beberapa faktor, di antaranya:

1.   Ekstraksi fitur HOG bergantung pada gradien citra, sementara ekstraksi fitur SIFT dan LBP didasarkan pada keberadaan fitur-fitur lokal seperti tepi atau tekstur. Keterampilan pemrosesan citra ini mungkin berbeda-beda dalam kemampuan untuk membedakan antara kucing dan anjing, tergantung pada kualitas dataset dan variasi antara kucing dan anjing.
2.   HOG menghasilkan deskripsi fitur yang lebih besar daripada SIFT atau LBP. Hal ini dapat memungkinkan SVM untuk mempelajari representasi fitur yang lebih baik dan dapat membedakan antara kelas kucing dan anjing dengan lebih baik. Ini juga ditandai dengan lamanya waktu saat melatih model dan memprediksinya menggunakan HOG.
3.   Model SVM menggunakan pendekatan pembelajaran mesin yang berbeda dari model CNN, tetapi dalam kasus ini, SVM mungkin lebih cocok untuk dataset ini karena jumlah sampel yang relatif kecil dan kemampuan SVM untuk mengatasi masalah klasifikasi biner.

Namun, perlu diingat bahwa hasil akurasi yang diperoleh mungkin tergantung pada parameter model dan parameter ekstraksi fitur yang digunakan, serta kualitas dataset yang digunakan. Oleh karena itu, perlu dilakukan pengujian lebih lanjut untuk menentukan metode ekstraksi fitur dan model yang paling cocok untuk dataset ini.